In [1]:
!pip install git+https://github.com/ismrmrd/ismrmrd-python-tools.git


  Cloning https://github.com/ismrmrd/ismrmrd-python-tools.git to /tmp/pip-req-build-s8oes3_2
  Running command git clone -q https://github.com/ismrmrd/ismrmrd-python-tools.git /tmp/pip-req-build-s8oes3_2


Mount Google Drive for access to data

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import ismrmrdtools
import numpy as np

In [5]:
train_name = os.listdir('/content/drive/My Drive/singlecoil_mini_train')

In [6]:
print(train_name)

['file1000053.h5', 'file1000060.h5', 'file1000005.h5', 'file1000032.h5', 'file1000015.h5', 'file1000012.h5', 'file1000043.h5', 'file1000001.h5', 'file1000023.h5', 'file1000027.h5', 'file1000059.h5', 'file1000057.h5', 'file1000021.h5', 'file1000045.h5', 'file1000048.h5', 'file1000039.h5', 'file1000003.h5', 'file1000029.h5', 'file1000002.h5', 'file1000040.h5', 'file1000064.h5', 'file1000058.h5', 'file1000061.h5', 'file1000010.h5', 'file1000728.h5', 'file1000723.h5', 'file1000736.h5', 'file1000734.h5', 'file1000740.h5', 'file1000738.h5', 'file1000750.h5', 'file1000743.h5', 'file1000742.h5', 'file1000753.h5', 'file1000752.h5', 'file1000760.h5', 'file1000755.h5', 'file1000754.h5', 'file1000768.h5', 'file1000762.h5', 'file1000777.h5', 'file1000776.h5', 'file1000770.h5', 'file1000785.h5', 'file1000784.h5', 'file1000778.h5', 'file1000795.h5', 'file1000794.h5', 'file1000790.h5', 'file1000806.h5', 'file1000801.h5', 'file1000814.h5', 'file1000808.h5', 'file1000807.h5', 'file1000824.h5', 'file1000

In [7]:
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

In [8]:
from ismrmrdtools import show, transform


Import tools for professing H5 file type

In [9]:
import h5py
import matplotlib.pyplot as plt  
import tensorflow as tf

Write function to access, crop, and convert data to tensors.
Each MRI includes many slices from a 3D scan, so func. chooses a random slice from each MRI to create more robust dataset.

In [10]:

#directory_name = train_name

def arrange_data(directory_name):
  data = []
  names = os.listdir('/content/drive/My Drive/'+directory_name)
  for k, name in enumerate(names):
    path = '/content/drive/My Drive/' + directory_name + '/' + name#singlecoil_mini_train/' +name
    ex =  h5py.File(path,'r')
    im = transform.transform_kspace_to_image(ex['kspace'])
    ind = np.random.randint(0,len(im))
    img = np.abs(im[ind])
    img = tf.constant(img)
    img = tf.expand_dims(img, 2) 
    img = tf.image.resize_with_pad(img,512,256)
    data += [img]
  return data


Write function to add random motion blur to images

In [11]:
pip install imgaug

In [12]:
from imgaug.augmenters import MotionBlur

In [13]:
def blur_data(unblurred):
  blurred_ims = []
  for t in unblurred:
    aug = MotionBlur(k=(5,10))
    blurred = aug.augment_image(t)
    blurred_ims += [tf.convert_to_tensor(blurred)]
  return blurred_ims

Get training data

In [14]:
mini_train = arrange_data('singlecoil_mini_train')
mini_train_blur = blur_data(mini_train)
#Convert to tensor
x_train = tf.stack(mini_train_blur)
y_train = tf.stack(mini_train)

Get validation data

In [15]:
mini_val = arrange_data('singlecoil_mini_val')
mini_val_blur = blur_data(mini_val)
#Convert to tensor
x_val = tf.stack(mini_val_blur)
y_val = tf.stack(mini_val)

In [16]:
pip install segmentation-models

In [17]:
import tensorflow as tf
import segmentation_models
from segmentation_models import Unet
from keras.layers import Input, Conv2D, Conv2DTranspose
from keras.models import Model

Segmentation Models: using `keras` framework.


In [18]:

import segmentation_models as sm

sm.set_framework('tf.keras')

sm.framework()

'tf.keras'

Adapt implementation of UNet to change channels and add deconvolution layer:[link text](https://)

In [19]:
x, y = mini_train_blur,mini_train
N = 1

base_model = sm.Unet(backbone_name='resnet34', encoder_weights=None,input_shape=(None, None, 1))#,input_shape=(None,None,1))
inp = Input(shape=(None,None,1))
l1 = Conv2DTranspose(1, (10, 10),padding='same')(inp)
out = base_model(l1)

model = Model(inp, out, name=base_model.name)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 1)]   0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, None, None, 1)    101       
 nspose)                                                         
                                                                 
 model_1 (Functional)        (None, None, None, 1)     24449876  
                                                                 
Total params: 24,449,977
Trainable params: 24,432,631
Non-trainable params: 17,346
_________________________________________________________________


In [21]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

Failed trial using jaccard loss

In [23]:
model.compile('Adam', loss=bce_jaccard_loss, metrics=[iou_score])


In [24]:
history = model.fit(x=x_train, y=y_train, epochs=15, batch_size=16, validation_data=(x_val, y_val))

Epoch 1/15
10/10 [==============================] - 462s 45s/step - loss: 1.6605 - iou_score: 2.0051e-05 - val_loss: 1441.1709 - val_iou_score: 6.2129e-09
Epoch 2/15
10/10 [==============================] - 463s 47s/step - loss: 1.3346 - iou_score: 1.9478e-05 - val_loss: 93.3927 - val_iou_score: 8.0165e-08
Epoch 3/15
10/10 [==============================] - 456s 46s/step - loss: 1.2150 - iou_score: 1.8481e-05 - val_loss: 4.5098 - val_iou_score: 6.2786e-07
Epoch 4/15
10/10 [==============================] - 437s 44s/step - loss: 1.1575 - iou_score: 1.9578e-05 - val_loss: 1.3787 - val_iou_score: 6.2786e-07
Epoch 5/15
10/10 [==============================] - 448s 45s/step - loss: 1.1201 - iou_score: 1.8368e-05 - val_loss: 1.0502 - val_iou_score: 6.2786e-07
Epoch 6/15
10/10 [==============================] - 452s 45s/step - loss: 1.0942 - iou_score: 1.8801e-05 - val_loss: 1.0188 - val_iou_score: 6.2706e-07
Epoch 7/15
10/10 [==============================] - 444s 44s/step - loss: 1.0760 - i

In [25]:
print(history.history)

{'loss': [1.6605173349380493, 1.334587812423706, 1.2150030136108398, 1.1574952602386475, 1.1200642585754395, 1.094154715538025, 1.076008677482605, 1.062649130821228, 1.052294373512268, 1.044215202331543, 1.0380502939224243, 1.0331230163574219, 1.02908194065094, 1.0257920026779175, 1.0229904651641846], 'iou_score': [2.0050709281349555e-05, 1.9477613022900186e-05, 1.8481299775885418e-05, 1.957792483153753e-05, 1.836826777434908e-05, 1.8800743418978527e-05, 1.901425093819853e-05, 1.8261203877045773e-05, 1.8523949620430358e-05, 1.856510243669618e-05, 1.8697533960221335e-05, 1.9200208043912426e-05, 1.8311076928512193e-05, 1.848841020546388e-05, 1.838880780269392e-05], 'val_loss': [1441.1708984375, 93.39274597167969, 4.509848117828369, 1.3786826133728027, 1.0502345561981201, 1.0188181400299072, 1.0086276531219482, 1.001765489578247, 1.0059340000152588, 1.0307432413101196, 1.065861463546753, 1.0888855457305908, 1.126776933670044, 1.1827600002288818, 1.2086387872695923], 'val_iou_score': [6.21

In [28]:
plt.figure()
plt.title("Loss")
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'][1:])
plt.title("Loss outcomes with Jaccard Loss")
plt.legend(["Training Loss","Validation Loss"])
plt.xlabel('Epoch')
plt.ylabel('Loss')

Text(0, 0.5, 'Loss')

In [29]:
from keras import losses

In [22]:
model.compile('Adam', loss='mse')

In [32]:
history2 = model.fit(x=x_train, y=y_train, validation_data=(x_val, y_val))

5/5 [==============================] - 466s 89s/step - loss: 4.2860e-04 - mse: 4.2860e-04 - val_loss: 0.0017 - val_mse: 0.0017


In [23]:
history2 = model.fit(x=x_train, y=y_train,epochs=10,batch_size=8, validation_data=(x_val, y_val))

Epoch 1/10
20/20 [==============================] - 479s 23s/step - loss: 0.0428 - val_loss: 0.0355
Epoch 2/10
20/20 [==============================] - 458s 23s/step - loss: 0.0043 - val_loss: 0.0035
Epoch 3/10
20/20 [==============================] - 461s 23s/step - loss: 0.0018 - val_loss: 0.0057
Epoch 4/10
20/20 [==============================] - 474s 24s/step - loss: 0.0011 - val_loss: 0.0033
Epoch 5/10
20/20 [==============================] - 463s 23s/step - loss: 8.3083e-04 - val_loss: 0.0171
Epoch 6/10
20/20 [==============================] - 477s 24s/step - loss: 6.4842e-04 - val_loss: 0.0073
Epoch 7/10
20/20 [==============================] - 472s 24s/step - loss: 5.1979e-04 - val_loss: 0.0046
Epoch 8/10
20/20 [==============================] - 471s 24s/step - loss: 4.3259e-04 - val_loss: 0.0028
Epoch 9/10
20/20 [==============================] - 469s 23s/step - loss: 3.6900e-04 - val_loss: 0.0018
Epoch 10/10
20/20 [==============================] - 481s 24s/step - loss: 3.149

In [24]:
print(history2.history)

{'loss': [0.04284447431564331, 0.004291392397135496, 0.0017824409296736121, 0.0011274849530309439, 0.00083082978380844, 0.0006484241457656026, 0.0005197909194976091, 0.0004325881600379944, 0.0003689951845444739, 0.00031496313749812543], 'val_loss': [0.03553720563650131, 0.003471099305897951, 0.005676369182765484, 0.0032779499888420105, 0.017092006281018257, 0.007329529616981745, 0.00457750353962183, 0.002760583069175482, 0.0018024432938545942, 0.0012572770938277245]}


In [26]:
plt.figure(3)
plt.title("Training and Validation Loss")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(history2.history['loss'],label='Training Loss')
plt.plot(history2.history['val_loss'],label='Validation Loss')
plt.legend()

Load test data from Drive

In [32]:
mini_test = arrange_data('mini_test_set')
mini_test_blur = blur_data(mini_test)
#Convert to tensor
x_test = tf.stack(mini_test_blur)
y_test = tf.stack(mini_test)

Evaluate with model

In [80]:
ev = model.evaluate(x_test,y_test)

1/1 [==============================] - 7s 7s/step - loss: 0.0013


In [81]:
print(ev)

0.001256986171938479
